In [32]:
import sympy as sp

# Step 1: Define symbolic variables for joint angles (q) and velocities (dq)
q1, q2, q3 = sp.symbols('q1 q2 q3')  # Joint angles
dq1, dq2, dq3 = sp.symbols('dq1 dq2 dq3')  # Joint velocities

# Define other parameters like link lengths, masses, and inertia
m1, m2, m3 = sp.symbols('m1 m2 m3')  # Mass of links 1, 2, and 3
l1, l2, l3 = sp.symbols('l1 l2 l3')  # Lengths of links 1, 2, and 3
I1, I2, I3 = sp.symbols('I1 I2 I3')  # Inertia of links 1, 2, and 3

# Step 2: Define the inertia matrix M(q) for the 3-DOF planar robot
M11 = I1 + I2 + I3 + m2 * l1**2 + m3 * (l1**2 + l2**2 + 2*l1*l2*sp.cos(q2)) + m3 * l3**2
M12 = I2 + I3 + m3 * (l2**2 + l1 * l2 * sp.cos(q2)) + m3 * l3**2
M13 = I3 + m3 * l3**2
M21 = M12  # Symmetric
M22 = I2 + I3 + m3 * l2**2 + m3 * l3**2
M23 = I3 + m3 * l3**2
M31 = M13  # Symmetric
M32 = M23  # Symmetric
M33 = I3 + m3 * l3**2

# Create the inertia matrix M
M = sp.Matrix([[M11, M12, M13],
               [M21, M22, M23],
               [M31, M32, M33]])

# Step 3: Compute the Coriolis matrix C(q, dq)

# Initialize the Coriolis matrix as a 3x3 matrix
C = sp.Matrix([[0, 0, 0],
               [0, 0, 0],
               [0, 0, 0]])

# Define the joint velocity vector
dq = sp.Matrix([dq1, dq2, dq3])

# Compute the Coriolis matrix
for i in range(3):
    for j in range(3):
        C_ij = 0  # Initialize C_ij to zero
        for k in range(3):
            # Compute each element of the Coriolis matrix
            C_ij += (1 / 2) * (
                sp.diff(M[i, j], [q1, q2, q3][k]) +
                sp.diff(M[i, k], [q1, q2, q3][j]) -
                sp.diff(M[j, k], [q1, q2, q3][i])
            ) * dq[k]
        
        # Assign the computed value to C[i, j]
        C[i, j] = C_ij

# Display the resulting Coriolis matrix
sp.pprint(C)
sp.pprint(M[0,2])


⎡-1.0⋅dq₂⋅l₁⋅l₂⋅m₃⋅sin(q₂)  -1.0⋅dq₁⋅l₁⋅l₂⋅m₃⋅sin(q₂) - 1.0⋅dq₂⋅l₁⋅l₂⋅m₃⋅sin(q ↪
⎢                                                                              ↪
⎢1.0⋅dq₁⋅l₁⋅l₂⋅m₃⋅sin(q₂)                            0                         ↪
⎢                                                                              ↪
⎣            0                                       0                         ↪

↪ ₂)  0⎤
↪      ⎥
↪     0⎥
↪      ⎥
↪     0⎦
       2   
I₃ + l₃ ⋅m₃


In [34]:
import sympy as sp

# Step 1: Define symbolic variables for joint angles (q) and velocities (dq)
q1, q2, q3 = sp.symbols('q1 q2 q3')  # Joint angles
dq1, dq2, dq3 = sp.symbols('dq1 dq2 dq3')  # Joint velocities

# Define other parameters like link lengths, masses, and inertia
m1, m2, m3 = sp.symbols('m1 m2 m3')  # Mass of links 1, 2, and 3
l1, l2, l3 = sp.symbols('l1 l2 l3')  # Lengths of links 1, 2, and 3
I1, I2, I3 = sp.symbols('I1 I2 I3')  # Inertia of links 1, 2, and 3

def inertia_matrix():
    M = sp.Matrix(3, 3, lambda i, j: 0)  # Initialize 3x3 matrix with zeros

    # Distances to centers of mass
    d1 = l1 / 2  # Distance to COM of link 1
    d2 = l1 + (l2 / 2) * sp.cos(q2)  # Distance to COM of link 2
    d3 = l1 + l2 * sp.cos(q2) + (l3 / 2) * sp.cos(q2 + q3)  # Distance to COM of link 3

    # M11 calculation
    M[0, 0] = I1 + m1 * d1**2 + m2 * d2**2 + m3 * d3**2

    # M22 calculation
    M[1, 1] = I2 + m2 * (l2 / 2)**2 + m3 * (l2 * sp.cos(q2) + (l3 / 2) * sp.cos(q2 + q3))**2

    # M33 calculation
    M[2, 2] = I3 + m3 * (l3 / 2)**2

    # Off-diagonal terms
    M[0, 1] = M[1, 0] = m2 * d2 + m3 * d3
    M[0, 2] = M[2, 0] = m3 * d3
    M[1, 2] = M[2, 1] = m3

    return M

⎡                                                                              ↪
⎢                    2                          2                          2   ↪
⎢0.583333333333333⋅l₁ ⋅m₁ + 0.333333333333333⋅l₂ ⋅m₂ + 0.333333333333333⋅l₃ ⋅m ↪
⎣                                                                              ↪

↪        ⎛                        2⎞                                           ↪
↪        ⎜  2                   l₂ ⎟      ⎛  2                       2     2   ↪
↪ ₃ + m₂⋅⎜l₁  + l₁⋅l₂⋅cos(q₂) + ───⎟ + m₃⋅⎜l₁  + 2⋅l₁⋅l₂⋅cos(q₂) + l₂  + l₃  + ↪
↪        ⎝                       4 ⎠      ⎝                                    ↪

↪                                                                              ↪
↪  l₃⋅(2⋅l₁ + 2⋅l₂)⋅cos(q₂ + q₃)⎞                                              ↪
↪  ─────────────────────────────⎟  l₁⋅m₂⋅cos(q₂) + m₃⋅(l₁⋅cos(q₂) + l₂⋅cos(q₂  ↪
↪                2              ⎠                                              ↪

↪                        

In [38]:
# Initialize the Coriolis matrix as a 3x3 matrix
C = sp.Matrix([[0, 0, 0],
               [0, 0, 0],
               [0, 0, 0]])

# Define the joint velocity vector
dq = sp.Matrix([dq1, dq2, dq3])

# Compute the Coriolis matrix
for i in range(3):
    for j in range(3):
        C_ij = 0  # Initialize C_ij to zero
        for k in range(3):
            # Compute each element of the Coriolis matrix
            C_ij += (1 / 2) * (
                sp.diff(M[i, j], [q1, q2, q3][k]) +
                sp.diff(M[i, k], [q1, q2, q3][j]) -
                sp.diff(M[j, k], [q1, q2, q3][i])
            ) * dq[k]
        
        # Assign the computed value to C[i, j]
        C[i, j] = C_ij

# Display the resulting Coriolis matrix
sp.pprint(C[0,:])


⎡    ⎛                               ⎛                   l₃⋅(2⋅l₁ + 2⋅l₂)⋅sin( ↪
⎢dq₂⋅⎜-0.5⋅l₁⋅l₂⋅m₂⋅sin(q₂) + 0.5⋅m₃⋅⎜-2⋅l₁⋅l₂⋅sin(q₂) - ───────────────────── ↪
⎣    ⎝                               ⎝                                 2       ↪

↪ q₂ + q₃)⎞⎞                                                  ⎛                ↪
↪ ────────⎟⎟ - 0.25⋅dq₃⋅l₃⋅m₃⋅(2⋅l₁ + 2⋅l₂)⋅sin(q₂ + q₃)  dq₁⋅⎜-0.5⋅l₁⋅l₂⋅m₂⋅s ↪
↪         ⎠⎠                                                  ⎝                ↪

↪                 ⎛                   l₃⋅(2⋅l₁ + 2⋅l₂)⋅sin(q₂ + q₃)⎞⎞          ↪
↪ in(q₂) + 0.5⋅m₃⋅⎜-2⋅l₁⋅l₂⋅sin(q₂) - ─────────────────────────────⎟⎟ + dq₂⋅(- ↪
↪                 ⎝                                 2              ⎠⎠          ↪

↪                                                                              ↪
↪ 1.0⋅l₁⋅m₂⋅sin(q₂) + 1.0⋅m₃⋅(-l₁⋅sin(q₂) - l₂⋅sin(q₂ + q₃))) + dq₃⋅(-0.5⋅l₂⋅m ↪
↪                                                                              ↪

↪                       